In [1]:
import pandas as pd

#load data from TSV

new_df= pd.read_csv('tsv/cumul_high.tsv', delimiter='\t')

display(new_df)

,freq_start,freq_stop,time_start,time_stop,amp_min,amp_max
0,31.25,97.368421,0.0,15.0,0.185020,0.127585
1,75.00,105.263158,0.0,15.0,0.283141,0.730145
2,127.50,110.526316,0.0,15.0,0.175785,0.320386
3,127.50,194.736842,0.0,15.0,0.175785,0.178615
4,238.75,202.631579,0.0,15.0,1.000000,0.465142
5,238.75,210.526316,0.0,15.0,1.000000,0.979685
6,238.75,215.789474,0.0,15.0,1.000000,0.722938
7,321.25,223.684211,0.0,15.0,0.803007,0.655112
8,321.25,228.947368,0.0,15.0,0.803007,0.415782
9,321.25,292.105263,0.0,15.0,0.803007,0.166797


In [2]:
def estimate_sampling_frequency_and_time_vector_updated(new_df):
    # Estimating the sampling frequency based on the highest frequency in the df
    highest_frequency = max(new_df['freq_start'].max(), new_df['freq_stop'].max())
    sampling_frequency = 2 * highest_frequency
    
    # Use the highest value from the time_stop column as wave_stop_value
    wave_stop_value = new_df['time_stop'].max()
    
    # Calculate the sampling interval
    delta_t = 1 / sampling_frequency
    
    return sampling_frequency, delta_t, wave_stop_value

# Using the updated function to estimate the sampling frequency and time vector
sampling_frequency_updated, delta_t_updated, duration = estimate_sampling_frequency_and_time_vector_updated(new_df)

# Printing the results
print(f"Estimated Optimal Sampling Frequency: {sampling_frequency_updated:.3f} Hz")
print(f"Sampling Interval (Delta t): {delta_t_updated:.6f} seconds")
print(f"Duration (highest value from time_stop column): {duration:.3f} seconds")


Estimated Optimal Sampling Frequency: 6110.526 Hz
Sampling Interval (Delta t): 0.000164 seconds
Duration (highest value from time_stop column): 15.000 seconds


In [3]:
from py_scripts.plot_wave import plot_waves

# define the time vector (from 0 to n seconds, with a step of k seconds)
k=0.00016

y_combined = plot_waves(new_df, k, disable_freq_plot=True, disable_amp_plot=True, disable_combined_plot=True, disable_wave_plot=True)

In [4]:
import numpy as np
from py_scripts.generate_wave_file import generate_wave_file
from IPython.display import Audio, display

player = True
save_audio = False

y_combined_no_nan = np.nan_to_num(y_combined)

fs_initial = 1 / k  # The initial sampling rate is the inverse of the time step (k)

def display_audio_from_array(audio_data, sample_rate):
    """Display the audio player using audio data array."""
    display(Audio(audio_data, rate=sample_rate))

# Generate the wave or audio data
if save_audio:
    filename = generate_wave_file(y_combined_no_nan, fs_initial, fs_target_name='44.1kHz', bit_rate=24, save_to_file=True)
    if player:
        display(Audio(filename=filename))
else:
    audio_data, fs_target = generate_wave_file(y_combined_no_nan, fs_initial, fs_target_name='44.1kHz', bit_rate=24, save_to_file=False)
    if player:
        display_audio_from_array(audio_data, fs_target)